In [1]:
#Import libraries
import pandas as pd
import numpy as np
import re
from nltk import word_tokenize
#import nltk #only run first time
#nltk.download('punkt') #only run first time
#nltk.download('stopwords') #only run first time
#nltk.download('wordnet') #only run first time

In [2]:
#Load data from YouTube
df = pd.read_excel('response.xlsx', index_col=0)
df['snippet.description'] = df['snippet.description'].apply(str)
#df['snippet.description'].head
#df.dtypes

In [3]:
# 1. Tokenization
from nltk import word_tokenize

text = df.apply(lambda row: word_tokenize(row["snippet.description"]), axis=1)
#text = text.rstrip()
#text = re.sub(r'[^a-zA-Z]',' ', text)

#text

In [27]:
# Import nltk packages and string 
from nltk.corpus import stopwords
import string

# Define stopwords to exclude
stop = set(stopwords.words('portuguese'))
stop.update(("facebook","instagram","youtube","twitter", "live","canal", "siga", "follow", "então", "pra", "the", "►", "👉", "➭", "♡", "☑"))
#print(stop)

# Define punctuations to exclude and lemmatizer
exclude = set(string.punctuation)
#print(exclude)

# Import the lemmatizer from nltk
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()

# Define word cleaning function
def clean(text, stop):
    text = text.rstrip()
    punc_free = ''.join(i for i in text if i not in exclude)
    stop_free = " ".join([i for i in punc_free.lower().split() if((i not in stop) and (not i.isdigit()))])
#    print(stop_free)
    normalized = " ".join(lemma.lemmatize(i) for i in stop_free.split())      
    return normalized

# Clean the emails in df and print results
text_clean=[]
for text in df['snippet.description']:
    text_clean.append(clean(text, stop).split())    
#print(text_clean)


In [26]:
# Import the packages
import gensim
from gensim import corpora

# Define the dictionary
dictionary = corpora.Dictionary(text_clean)

# Define the corpus 
corpus = [dictionary.doc2bow(text) for text in text_clean]

# Print corpus and dictionary
#print(corpus)
#print(dictionary)

In [24]:
# Define the LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=5)

# Save the topics and top 5 words
topics = ldamodel.print_topics(num_words=5)

# Print the results
for topic in topics:
    print(topic)

(0, '0.009*"and" + 0.009*"stray" + 0.008*"xbox" + 0.008*"kid" + 0.007*"official"')
(1, '0.007*"aqui" + 0.006*"sobre" + 0.005*"produção" + 0.004*"banho" + 0.004*"dark"')
(2, '0.012*"séries" + 0.009*"melhores" + 0.009*"dark" + 0.007*"netflix" + 0.006*"casa"')
(3, '0.015*"maria" + 0.007*"tiringa" + 0.007*"loud" + 0.006*"vida" + 0.006*"nada"')
(4, '0.007*"vídeo" + 0.006*"aqui" + 0.006*"brasil" + 0.006*"pode" + 0.005*"profissões"')


In [25]:
import pyLDAvis.gensim

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)

pyLDAvis.display(lda_display)